# Applying Machine Learning on UrbanSound8k 

## Install Packages

We install: 
- Machine learning libraries: `tensorflow`, `sklearn`
- Audio processing: `librosa`
- Plots: `Plotly`, `matplotlib`

In [9]:
!pip install pandas
!pip install setuptools
!pip install numpy
!pip install sklearn
!pip install librosa
!pip install plotly
!pip install matplotlib
!pip install pillow
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━

In [1]:
# Unzip dataset
import os

if not os.path.isdir("UrbanSound8K"):
    !wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz
    !tar -xzf urban8k.tgz
    !rm urban8k.tgz


In [1]:
import os
import time
import librosa
import zipfile
import numpy as np
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2022-04-16 22:27:38.595860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-16 22:27:38.595936: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.8.0


## Design Choices and Models

Проанализировав набор данных и потратив немного времени на чтение о современном состоянии классификации аудиосигналов, я сделал следующие варианты дизайна и предложения:

Обучите свёрточную нейронную сеть и используйте в качестве входных данных MFCC, STFT или Mel-Spectogram.

- Поскольку продолжительность аудио варьируется от 0 до 4 с, я дополняю сгенерированную спектрограмму, чтобы сделать все аудио равной длины.

Варианты признаков:

Использование MFCC в качестве признаков:
   - Обычно вычисляют первые 13 MFCC, их производные и вторые производные и используют их в качестве признаков.
   - Или также обычно используется 40 MFCC, так как это значение по умолчанию для Librosa.

- Использование STFT в качестве признаков:
   - Содержит меньше человеческой обработки, чем MFCC и Mel-Spectogram, CNN может изучать другие фильтры, а не представления, разработанные людьми.

- Использование Mel-Spectogram в качестве признаков:
   - Преобразование, примененное к STFT, которое приблизительно соответствует тому, как люди воспринимают звук. Немного проще чем MFCC, но немного сложнее чем STFT.

Моим первым выбором было бы использование STFT и Mel-Spectogram, поскольку похоже, что CNN могут использовать больше преимуществ частотно-временной структуры, но из-за **вычислительных ресурсов** и ограниченного времени я покажу использование **MFCC** в качестве функций. поскольку они намного более эффективны с точки зрения памяти.
 

## Dataset Preprocessing and Splits

Я загружаю все аудиоданные с помощью Librosa и частоты дискретизации по умолчанию 22050 Гц. Это дизайнерское решение основано на
([Источник](https://librosa.org/blog/2019/07/17/resample-on-load/#Okay...-but-why-22050-Hz?--Why-not-44100-or-48000?)) и в дальнейших экспериментах можно было попробовать разные частоты дискретизации.

> Люди могут слышать примерно до 20000 Гц, можно успешно анализировать музыкальные и речевые данные на гораздо более низких частотах без особых потерь. Самые высокие частоты, которые мы обычно стараемся обнаружить, составляют около C9 ≈ 8372 Гц, что значительно ниже порога 11025, подразумеваемого fs = 22050.

По умолчанию Librosa загружает звук в моно, давая нам 1 канал.



In [65]:
# FeatureExtractor class including librosa audio processing functions
class FeatureExtractor:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.max_audio_duration = 4
        self.dataset_df = self._create_dataset(csv_file)
    
    @staticmethod
    def _create_dataset(csv_file):
        """
        Args:
            dataset_path: path with the .wav files after unzipping
        Returns: A pandas dataframe with the list of files and labels (`filenames`, `labels`)
        """
        dataset_df = pd.read_csv(csv_file)
        filepaths = []
        for i, row in dataset_df.iterrows():
            filepaths.append(os.path.join('UrbanSound8K/audio', 'fold'+str(row['fold']), row['slice_file_name']))
        dataset_df['filepath'] = filepaths
        return dataset_df

    @staticmethod
    def _compute_max_pad_length(max_audio_length, sample_rate=22050, n_fft=2048, hop_length=512):
        dummy_file = np.random.random(max_audio_length*sample_rate)
        stft = librosa.stft(dummy_file, n_fft=n_fft, hop_length=hop_length)
        # Return an even number for CNN computation purposes
        if stft.shape[1] % 2 != 0:
            return stft.shape[1] + 1
        return stft.shape[1]

    def compute_save_features(self, 
                        mode='mfcc', 
                        sample_rate=22050,
                        n_fft=2048,
                        hop_length=512,
                        n_mfcc=40,
                        output_path='features',
                        deltas=False
                        ):
        dataset_features = []
        max_pad = self._compute_max_pad_length(self.max_audio_duration, 
                                               sample_rate=sample_rate, 
                                               n_fft=n_fft,
                                               hop_length=hop_length)
        print('Max Padding = ', max_pad)
        
        if not os.path.exists(output_path):
            print('Creating output folder: ', output_path)
            os.makedirs(output_path)
        else:
            print('Output folder already existed')
            
        print('Saving features in ', output_path)
        i = 0
        t = time.time()
        
        features_path = []
        for filepath in self.dataset_df['filepath']:
            if i % 100 == 0:
                print('{} files processed in {}s'.format(i, time.time() - t))
            audio_file, sample_rate = librosa.load(filepath, sr=sample_rate, res_type='kaiser_fast')
            if mode == 'mfcc':
                audio_features = self.compute_mfcc(audio_file, sample_rate, n_fft, hop_length, n_mfcc, deltas)  
            elif mode == 'stft':
                audio_features = self.compute_stft(audio_file, sample_rate, n_fft, hop_length)
            elif mode == 'mel-spectogram':
                audio_features = self.compute_mel_spectogram(audio_file, sample_rate, n_fft, hop_length)
            
            # print('audio_features',audio_features.shape)
            audio_features = np.pad(audio_features, 
                                    pad_width=((0, 0), (0, max_pad - audio_features.shape[1]),(0, 0)))
            
            # print('audio_features pad',audio_features.shape)
            
            save_path = os.path.join(output_path, filepath.split('/')[-1].replace('wav', 'npy'))
            self.save_features(audio_features, save_path)
            features_path.append(save_path)
            i+=1
        self.dataset_df['features_path'] = features_path
        return self.dataset_df
    
    @staticmethod
    def save_features(audio_features, filepath):
        np.save(filepath, audio_features)

    @staticmethod
    def compute_mel_spectogram(audio_file, sample_rate, n_fft, hop_length):
        return librosa.feature.melspectrogram(y=audio_file,
                                              sr=sample_rate, 
                                              n_fft=n_fft,
                                              hop_length=hop_length)
    @staticmethod
    def compute_stft(audio_file, sample_rate, n_fft, hop_length):
        return librosa.stft(audio_file, n_fft=n_fft, hop_length=hop_length)
    
    @staticmethod
    def compute_mfcc(audio_file, sample_rate, n_fft, hop_length, n_mfcc, deltas=False):
        mfccs = librosa.feature.mfcc(y=audio_file,
                                    sr=sample_rate, 
                                    n_fft=n_fft,
                                    n_mfcc=n_mfcc,
                                    )
        # Change mode from interpolation to nearest
        if deltas:
            mfccs = np.expand_dims(mfccs,axis=2)
            
            delta_mfccs = librosa.feature.delta(mfccs, mode='nearest')
            delta2_mfccs = librosa.feature.delta(mfccs, order=2, mode='nearest')
            
            return np.concatenate((mfccs, delta_mfccs, delta2_mfccs),axis=2)
        return mfccs

# Create dataset and extract features
fe = FeatureExtractor('UrbanSound8K/metadata/UrbanSound8K.csv')

In [67]:
#run to compute and save features on the colab notebook
if not os.path.isdir("features_mfcc"):
    dataset_df = fe.compute_save_features(mode='mfcc', n_mfcc=13, output_path='features_mfcc', deltas=True)
    dataset_df.to_json('dataset.json')

Max Padding =  174
Creating output folder:  features_mfcc
Saving features in  features_mfcc
0 files processed in 0.00031113624572753906s
100 files processed in 12.606899976730347s
200 files processed in 24.748304843902588s
300 files processed in 38.89931797981262s
400 files processed in 51.65732264518738s
500 files processed in 64.08351802825928s
600 files processed in 76.73673844337463s
700 files processed in 90.54266810417175s
800 files processed in 106.96380186080933s
900 files processed in 119.08151531219482s
1000 files processed in 130.72327971458435s
1100 files processed in 143.8911895751953s
1200 files processed in 157.42307019233704s
1300 files processed in 171.55621576309204s
1400 files processed in 184.84662318229675s
1500 files processed in 194.1484808921814s
1600 files processed in 208.56130170822144s
1700 files processed in 221.17717218399048s
1800 files processed in 234.45928263664246s
1900 files processed in 245.62158918380737s
2000 files processed in 256.79094409942627s

/home/dizorin/.local/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)


3600 files processed in 452.9502897262573s
3700 files processed in 465.0124680995941s
3800 files processed in 476.74525237083435s
3900 files processed in 490.2759349346161s
4000 files processed in 502.8731346130371s
4100 files processed in 515.9406824111938s
4200 files processed in 527.1015446186066s
4300 files processed in 538.9200437068939s
4400 files processed in 559.7939870357513s
4500 files processed in 571.1860523223877s
4600 files processed in 582.9600696563721s
4700 files processed in 596.495890378952s
4800 files processed in 609.6102838516235s
4900 files processed in 623.4087707996368s
5000 files processed in 639.0206425189972s
5100 files processed in 653.365775346756s
5200 files processed in 668.2914254665375s
5300 files processed in 683.5758645534515s
5400 files processed in 696.6333961486816s
5500 files processed in 708.2717821598053s
5600 files processed in 719.531896352768s
5700 files processed in 730.679196357727s
5800 files processed in 742.8220293521881s
5900 files pro

/home/dizorin/.local/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
/home/dizorin/.local/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)


8400 files processed in 1041.194732427597s
8500 files processed in 1053.2792451381683s
8600 files processed in 1065.676104068756s
8700 files processed in 1076.688779592514s


Доступ к диску и librosa загрузка аудиофайлов очень медленная на ноутбуке colab (30-40 минут), вместо этого мы могли бы загрузить предварительно вычисленные функции.

In [6]:
# Unzip features
if not os.path.isdir("features_mfcc"):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BU2B5EcbfyGBIOkB5YC44hpzPpuqw43H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BU2B5EcbfyGBIOkB5YC44hpzPpuqw43H" -O features_mfcc.zip && rm -rf /tmp/cookies.txt
    !unzip -q features_mfcc.zip
    !rm features_mfcc.zip

In [2]:
# Download dataset.json file
if not os.path.isfile("dataset.json"):
    !wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1pzSvGYaBXghLQFTZxlSex-Ts3T4B0X4C" -O dataset.json

dataset_df = pd.read_json('dataset.json')

Для целей этого эксперимента мы будем загружать все данные в память и обрабатывать их в мини-пакетах. Если бы у нас были вычислительные ресурсы и больше времени, мы могли бы создавать объекты Dataloader, которые позволяли бы выполнять многие другие операции, такие как увеличение данных, и выполнять итерации быстрее.

In [3]:
dataset_df['features'] = [np.asarray(np.load(feature_path)) for feature_path in dataset_df['features_path']]

from tensorflow.keras.utils import to_categorical
dataset_df['labels_categorical'] = [to_categorical(label, 10) for label in dataset_df['classID']]

dataset_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,filepath,features_path,features,labels_categorical
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,UrbanSound8K/audio/fold5/100032-3-0-0.wav,features_mfcc/100032-3-0-0.npy,"[[[-336.0325, 1.1658469e-15, -7.1054274e-15], ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-117.wav,features_mfcc/100263-2-0-117.npy,"[[[-492.85135, 1.7099216e-15, -7.1054274e-15],...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-121.wav,features_mfcc/100263-2-0-121.npy,"[[[-497.33685, 1.7254838e-15, -1.4210855e-14],...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-126.wav,features_mfcc/100263-2-0-126.npy,"[[[-448.51984, 1.5561158e-15, -7.1054274e-15],...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,UrbanSound8K/audio/fold5/100263-2-0-137.wav,features_mfcc/100263-2-0-137.npy,"[[[-474.92578, 1.6477298e-15, -7.1054274e-15],...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Мы собираемся создать разделения для обучающих, проверочных и тестовых наборов данных.
В целях эксперимента и для его ускорения мы будем использовать функцию sklearn `train_test_split` два раза.

In [4]:
# Split the dataset 
from sklearn.model_selection import train_test_split 

# Add one dimension for the channel
X = np.array(dataset_df['features'].tolist())
y = np.array(dataset_df['labels_categorical'].tolist())

# As there is unbalance for some classes I am going to stratify it so we have the same proportion in train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.30, 
                                                    random_state=1, 
                                                    stratify=y)
# Create validation and test
X_test, X_val, Y_test, Y_val = train_test_split(X_test, 
                                                Y_test, 
                                                test_size=0.5, 
                                                random_state=1, 
                                                stratify=Y_test)

norm1 = tf.keras.layers.Normalization()
norm1.adapt(X_train)

X_train = norm1(X_train)
X_val = norm1(X_val)
X_test = norm1(X_test)

print(X_train.shape, X_val.shape, X_test.shape)

2022-04-16 22:28:07.662862: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-04-16 22:28:07.662945: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ArchPC
2022-04-16 22:28:07.662961: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ArchPC
2022-04-16 22:28:07.663152: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 390.141.0
2022-04-16 22:28:07.663218: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.141.0
2022-04-16 22:28:07.663229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 390.141.0
2022-04-16 22:28:07.664550: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical ope

(6112, 13, 174, 3) (1310, 13, 174, 3) (1310, 13, 174, 3)


## Machine Learning Model

### Model Design

Мы собираемся создать модель **Fully Convolutional Network**, используя Keras, работающий поверх Tensorflow с несколькими слоями.

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, Conv2D, MaxPooling2D, BatchNormalization

Поскольку наши изображения имеют прямоугольную форму (ось y — это MFCC, ось x — время), вместо использования квадратных фильтров (как обычно) мы собираемся сделать их прямоугольными, чтобы они могли лучше изучить корреляцию MFCC с временным измерением.

In [14]:
# FCN Model
def create_model(num_classes=10, model_type=None, input_shape=None, dropout_ratio=None):
    model = Sequential()
    if input_shape is None:
        model.add(Input(shape=(None, None, 1)))
    else:
        model.add(Input(shape=input_shape))
        
    model.add(tf.keras.layers.RandomContrast(factor=(0.2,1.8))) #(x - mean) * factor + mean 
    model.add(tf.keras.layers.RandomTranslation(height_factor=0, width_factor=(-0.2, 0.2),fill_mode='constant'))
    model.add(tf.keras.layers.RandomZoom(height_factor=0, width_factor=(-0.2, 0.2),fill_mode='constant'))
    
    if model_type is None:
        model.add(Conv2D(filters=16, kernel_size=(2, 4), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 3)))
        model.add(Conv2D(filters=32, kernel_size=(2, 4), activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(filters=64, kernel_size=(2, 4), activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(filters=128, kernel_size=(2, 4), activation='relu'))
    else:
        #model.add(Conv2D(3,1))
        
        if model_type == 'VGG16':
            vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
            model.add(tf.keras.Model(inputs=vgg.get_layer("block1_conv1").input,
                                outputs=vgg.get_layer('block5_conv3').output))
            
        elif model_type == 'EfficientNetV2B1':
            eff = tf.keras.applications.EfficientNetV2B1(weights=None, include_top=False, classes=100)
            model.add(tf.keras.Model(inputs=eff.input,
                            outputs=eff.get_layer('top_activation').output))
            
        elif model_type == 'EfficientNetV2B0':
            eff0 = tf.keras.applications.EfficientNetV2B0(weights=None, include_top=False, classes=100)
            # eff0.summary()
            model.add(tf.keras.Model(inputs=eff0.input,
                            outputs=eff0.get_layer('top_activation').output))
            
        elif model_type == 'MobileNetV2':
            mb = tf.keras.applications.MobileNetV2(weights=None, include_top=False, classes=100)
            # mb.summary()
            model.add(tf.keras.Model(inputs=mb.input,
                            outputs=mb.output))
            
    model.add(Flatten())
    # model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5))
    
    # model.add(GlobalAveragePooling2D())
    # # model.add(Flatten())
    # if dropout_ratio is not None:
    #     model.add(Dropout(dropout_ratio))
    # Add dense linear layer
    model.add(Dense(num_classes, activation='softmax'))
    return model

Поскольку это проблема множественной классификации, мы будем использовать **Categorical Cross Entropy loss**. В качестве оптимизатора мы будем использовать реализацию Keras **Adam** со значениями гиперпараметров по умолчанию.

In [15]:
# Create and compile the model
fcn_model = create_model(input_shape=X_train.shape[1:], model_type='EfficientNetV2B0')
fcn_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
fcn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast_1 (RandomCo  (None, 13, 174, 3)       0         
 ntrast)                                                         
                                                                 
 random_translation_1 (Rando  (None, 13, 174, 3)       0         
 mTranslation)                                                   
                                                                 
 random_zoom_1 (RandomZoom)  (None, 13, 174, 3)        0         
                                                                 
 model_1 (Functional)        (None, None, None, 1280)  5919312   
                                                                 
 flatten_1 (Flatten)         (None, 7680)              0         
                                                                 
 dense_3 (Dense)             (None, 128)              

### Model training and evaluation

In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint 

In [ ]:
!mkdir saved_models

In [11]:
def train_model(model, X_train, Y_train, X_val, Y_val, epochs, batch_size, callbacks):
    history = model.fit(X_train, 
              Y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(X_val, Y_val), 
              callbacks=callbacks, verbose=1)
    
    fig, (ax0, ax1) = plt.subplots(1,2, figsize=(10,5))

    # print(list(history.history))
    ax0.plot(history.history['accuracy'])
    ax0.plot(history.history['val_accuracy'])
    ax0.set_title('model accuracy')
    ax0.set_ylabel('accuracy')
    ax0.set_xlabel('epoch')
    ax0.legend(['train', 'val'], loc='upper left')

    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('model loss')
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'val'], loc='upper left')

    plt.show()
    return model

Мы создадим контрольную точку для **ранней остановки**, поэтому мы выберем модель, которая лучше работает на проверочном наборе.

Создание функции для обучения модели позволит нам быстрее выполнять настройку гиперпараметров.

In [16]:
checkpointer = ModelCheckpoint(filepath='saved_models/best_fcn_eff2B0.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks = [checkpointer]

# fcn_model = load_model('saved_models/best_fcn.hdf5')
# fcn_model.summary()

# Hyper-parameters
epochs = 200
batch_size = 256

In [17]:
# Train the model
model = train_model(model=fcn_model,
                    X_train=X_train,
                    X_val=X_val,
                    Y_train=Y_train,
                    Y_val=Y_val,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=callbacks)

Epoch 1/200
24/24 [==============================] - ETA: 0s - loss: 2.4450 - accuracy: 0.1556
Epoch 1: val_accuracy improved from -inf to 0.10611, saving model to saved_models/best_fcn_eff2B0.hdf5
24/24 [==============================] - 115s 4s/step - loss: 2.4450 - accuracy: 0.1556 - val_loss: 2.3026 - val_accuracy: 0.1061
Epoch 2/200
24/24 [==============================] - ETA: 0s - loss: 2.2123 - accuracy: 0.1808
Epoch 2: val_accuracy did not improve from 0.10611
24/24 [==============================] - 97s 4s/step - loss: 2.2123 - accuracy: 0.1808 - val_loss: 2.3477 - val_accuracy: 0.0840
Epoch 3/200
24/24 [==============================] - ETA: 0s - loss: 2.1366 - accuracy: 0.2063
Epoch 3: val_accuracy did not improve from 0.10611
24/24 [==============================] - 96s 4s/step - loss: 2.1366 - accuracy: 0.2063 - val_loss: 2.6880 - val_accuracy: 0.0557
Epoch 4/200
24/24 [==============================] - ETA: 0s - loss: 2.1032 - accuracy: 0.2246
Epoch 4: val_accuracy did n

KeyboardInterrupt: 

In [46]:
# Load the best model
best_model = load_model('saved_models/best_fcn_none.hdf5')

# Evaluating the model on the training and testing set
score = best_model.evaluate(X_train, Y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = best_model.evaluate(X_val, Y_val, verbose=0)
print("Validation Accuracy: ", score[1])

score = best_model.evaluate(X_test, Y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9874018430709839
Validation Accuracy:  0.9297710061073303
Testing Accuracy:  0.9137404561042786


Похоже, что модель переоснащена тренировочными данными к концу обучения. Мы выбрали модель, которая показала лучшие результаты на проверочном наборе, сохраненном контрольной точкой. Сходство между проверкой и тестовой оценкой говорит нам о том, что наша методология обучения верна и что наш проверочный набор является хорошей оценкой производительности тестирования.

In [ ]:
# Plot a confusion matrix
from sklearn import metrics
Y_pred = best_model.predict(X_test)
matrix = metrics.confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1))

In [ ]:
# Confusion matrix code (from https://github.com/triagemd/keras-eval/blob/master/keras_eval/visualizer.py)
def plot_confusion_matrix(cm, concepts, normalize=False, show_text=True, fontsize=18, figsize=(16, 12),
                          cmap=plt.cm.coolwarm_r, save_path=None, show_labels=True):
    '''
    Plot confusion matrix provided in 'cm'
    Args:
        cm: Confusion Matrix, square sized numpy array
        concepts: Name of the categories to show
        normalize: If True, normalize values between 0 and ones. Not valid if negative values.
        show_text: If True, display cell values as text. Otherwise only display cell colors.
        fontsize: Size of text
        figsize: Size of figure
        cmap: Color choice
        save_path: If `save_path` specified, save confusion matrix in that location
    Returns: Nothing. Plots confusion matrix
    '''

    if cm.ndim != 2 or cm.shape[0] != cm.shape[1]:
        raise ValueError('Invalid confusion matrix shape, it should be square and ndim=2')

    if cm.shape[0] != len(concepts) or cm.shape[1] != len(concepts):
        raise ValueError('Number of concepts (%i) and dimensions of confusion matrix do not coincide (%i, %i)' %
                         (len(concepts), cm.shape[0], cm.shape[1]))

    plt.rcParams.update({'font.size': fontsize})

    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if normalize:
        cm = cm_normalized

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm, vmin=np.min(cm), vmax=np.max(cm), alpha=0.8, cmap=cmap)

    fig.colorbar(cax)
    ax.xaxis.tick_bottom()
    plt.ylabel('True label', fontweight='bold')
    plt.xlabel('Predicted label', fontweight='bold')

    if show_labels:
        n_labels = len(concepts)
        ax.set_xticklabels(concepts)
        ax.set_yticklabels(concepts)
        plt.xticks(np.arange(0, n_labels, 1.0), rotation='vertical')
        plt.yticks(np.arange(0, n_labels, 1.0))
    else:
        plt.axis('off')

    if show_text:
        # http://stackoverflow.com/questions/21712047/matplotlib-imshow-matshow-display-values-on-plot
        min_val, max_val = 0, len(concepts)
        ind_array = np.arange(min_val, max_val, 1.0)
        x, y = np.meshgrid(ind_array, ind_array)
        for i, (x_val, y_val) in enumerate(zip(x.flatten(), y.flatten())):
            c = cm[int(x_val), int(y_val)]
            ax.text(y_val, x_val, c, va='center', ha='center')

    if save_path is not None:
        plt.savefig(save_path)

Чтобы лучше наблюдать за производительностью модели и ошибками, допущенными между разными классами, мы строим матрицу путаницы.

В нашем случае точность является хорошей метрикой, потому что набор данных в основном сбалансирован, но мы наблюдали несколько классов с меньшим количеством выборок (1 «car_horn», «gun_shot» и «siren»), поэтому будет хорошо наблюдать за производительностью этих классов. .

Мы можем заметить, что между классом Children_playing и классом street_music происходит много ошибок, поэтому, возможно, стоит потратить немного больше времени на анализ и поиск возможных причин.

In [ ]:
class_dictionary = {3: 'dog_bark', 2: 'children_playing', 1: 'car_horn', 0: 'air_conditioner', 9: 'street_music', 6: 'gun_shot', 8: 'siren', 5: 'engine_idling', 7: 'jackhammer', 4: 'drilling'}
classes = [class_dictionary[key] for key in sorted(class_dictionary.keys())]

plot_confusion_matrix(matrix, classes)

## Conclusions

Мы можем наблюдать увеличение точности набора тестов на 1-2% при введении отсева в качестве регуляризации. Это показывает, что это было успешное дополнение к нашей модели.

Есть много вещей, которые мы можем попробовать улучшить производительность модели, например:

- Настройка гиперпараметров:
   - Настройка параметров извлечения признаков
   - Настройка параметров сети (количество слоев, объединение слоев, количество и форма фильтра...)
   - Настройка гиперпараметров сети (скорость обучения, оптимизатор)

- Извлечение признаков:
   - Используйте STFT: необработанная спектрограмма может предоставить CNN больше информации для изучения корреляции между частотой и временем, чем MFCC.
   - Используйте мел-спектограмму: Мел-спектограмма может предоставить CNN больше информации для изучения корреляции между частотой и временем, чем MFCC.